<a href="https://colab.research.google.com/github/jasanchez19/Machine-Learning-Create-Neural-Networks/blob/main/Homework6JorgeSanchezECGR4105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

#Problem 1

Part A

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out


In [6]:
import datetime
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    print('{} Epoch {}, Training loss {}'.format(
    datetime.datetime.now(), epoch,
    loss_train / len(train_loader)))

In [7]:
def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
          imgs, labels = imgs.to(device), labels.to(device)
          batchsize = imgs.shape[0]
          outputs = model(imgs)
          _, predicted = torch.max(outputs, dim=1)
          total += labels.shape[0]
          correct += int((predicted == labels).sum())
    print("Accuracy {}: {:.2f}".format(name , correct / total))

In [8]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)  # <1>

model = Net().to(device=device)  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)


2022-12-07 16:45:57.743996 Epoch 1, Training loss 2.027217293029551
2022-12-07 16:46:10.069563 Epoch 2, Training loss 1.7497720678748987
2022-12-07 16:46:22.445291 Epoch 3, Training loss 1.5772512189262664
2022-12-07 16:46:34.823357 Epoch 4, Training loss 1.4791934827099675
2022-12-07 16:46:47.174110 Epoch 5, Training loss 1.407292333405341
2022-12-07 16:46:59.446634 Epoch 6, Training loss 1.3408779095658256
2022-12-07 16:47:11.808416 Epoch 7, Training loss 1.281713824214228
2022-12-07 16:47:24.160779 Epoch 8, Training loss 1.2311879630436373
2022-12-07 16:47:36.550405 Epoch 9, Training loss 1.1906362339053922
2022-12-07 16:47:49.062809 Epoch 10, Training loss 1.1571512606442738
2022-12-07 16:48:01.599214 Epoch 11, Training loss 1.1288810272503387
2022-12-07 16:48:14.085641 Epoch 12, Training loss 1.1048441008686105
2022-12-07 16:48:26.641470 Epoch 13, Training loss 1.0834906156867972
2022-12-07 16:48:39.093302 Epoch 14, Training loss 1.0671912920292077
2022-12-07 16:48:51.199261 Epoch

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)


Accuracy train: 0.81
Accuracy val: 0.62


Part B

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(8, 3, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out


In [11]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                        shuffle=True)
model = Net().to(device=device)  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-12-07 17:50:31.445862 Epoch 1, Training loss 2.0514538217993343
2022-12-07 17:50:43.460893 Epoch 2, Training loss 1.7980358996964476
2022-12-07 17:50:55.384909 Epoch 3, Training loss 1.6203605169835298
2022-12-07 17:51:07.440745 Epoch 4, Training loss 1.5030134390382206
2022-12-07 17:51:19.458525 Epoch 5, Training loss 1.415657628070363
2022-12-07 17:51:31.457748 Epoch 6, Training loss 1.3448547268919933
2022-12-07 17:51:43.506264 Epoch 7, Training loss 1.2895419903271033
2022-12-07 17:51:55.412668 Epoch 8, Training loss 1.2466653861353159
2022-12-07 17:52:07.502946 Epoch 9, Training loss 1.211783008349826
2022-12-07 17:52:19.485119 Epoch 10, Training loss 1.1828700975536386
2022-12-07 17:52:31.547303 Epoch 11, Training loss 1.1574098905334083
2022-12-07 17:52:43.601510 Epoch 12, Training loss 1.1359829952192428
2022-12-07 17:52:55.486710 Epoch 13, Training loss 1.117091161744369
2022-12-07 17:53:07.547972 Epoch 14, Training loss 1.0998158029583105
2022-12-07 17:53:19.431162 Epoch

In [12]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)

Accuracy train: 0.76
Accuracy val: 0.60


#Problem 2

Part A

In [13]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
                               padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        out = out.view(-1, 4 * 4 * self.n_chans1 // 2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [14]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
model = NetRes(n_chans1=32).to(device=device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-12-07 19:33:56.468503 Epoch 1, Training loss 2.0726132075804884
2022-12-07 19:34:08.964743 Epoch 2, Training loss 1.7108220932123912
2022-12-07 19:34:21.501335 Epoch 3, Training loss 1.522366407277334
2022-12-07 19:34:33.854186 Epoch 4, Training loss 1.4175503389609745
2022-12-07 19:34:46.402777 Epoch 5, Training loss 1.3382294338834866
2022-12-07 19:34:58.755141 Epoch 6, Training loss 1.2684047984345186
2022-12-07 19:35:11.276311 Epoch 7, Training loss 1.211839218959784
2022-12-07 19:35:23.592098 Epoch 8, Training loss 1.1649688265055342
2022-12-07 19:35:35.995739 Epoch 9, Training loss 1.121424007674922
2022-12-07 19:35:48.358969 Epoch 10, Training loss 1.0826403432337524
2022-12-07 19:36:00.640971 Epoch 11, Training loss 1.0535673619536183
2022-12-07 19:36:13.113313 Epoch 12, Training loss 1.0236433068352282
2022-12-07 19:36:25.554658 Epoch 13, Training loss 0.9989350533393948
2022-12-07 19:36:38.194206 Epoch 14, Training loss 0.9772031695184196
2022-12-07 19:36:50.573838 Epoch

In [15]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)

Accuracy train: 0.84
Accuracy val: 0.68


Part B, normalize batch conv layer

In [16]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
          super(NetRes, self).__init__()
          self.conv = nn.Conv2d(n_chans1, n_chans1, 
            kernel_size=3, padding=1, bias=False)
          self.batch_norm = nn.BatchNorm2d(num_features=
                                     n_chans1)
          torch.nn.init.kaiming_normal_(self.conv.weight, 
                            nonlinearity='relu')
          torch.nn.init.constant_(self.batch_norm.weight, 
                            0.5)
          torch.nn.init.zeros_(self.batch_norm.bias)
        
    def forward(self, x):
     out = self.conv(x)
     out = torch.relu(out)
     return out + x


In [17]:
train_loader = torch.utils.data.DataLoader(cifar10, 
                        batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)


2022-12-07 20:38:29.457445 Epoch 1, Training loss 2.0330074699333562
2022-12-07 20:38:41.250285 Epoch 2, Training loss 1.730403132450855
2022-12-07 20:38:53.138689 Epoch 3, Training loss 1.5619641362553667
2022-12-07 20:39:04.994569 Epoch 4, Training loss 1.468449181607922
2022-12-07 20:39:16.809624 Epoch 5, Training loss 1.396753173807393
2022-12-07 20:39:28.565919 Epoch 6, Training loss 1.3324053040550798
2022-12-07 20:39:40.348994 Epoch 7, Training loss 1.2726079954972962
2022-12-07 20:39:52.166385 Epoch 8, Training loss 1.2255400512224572
2022-12-07 20:40:04.017866 Epoch 9, Training loss 1.188900394970194
2022-12-07 20:40:15.836572 Epoch 10, Training loss 1.1564261196062082
2022-12-07 20:40:27.665418 Epoch 11, Training loss 1.1301433671923244
2022-12-07 20:40:39.464533 Epoch 12, Training loss 1.1069394989544168
2022-12-07 20:40:51.313287 Epoch 13, Training loss 1.085099760681162
2022-12-07 20:41:03.135575 Epoch 14, Training loss 1.0665156776490419
2022-12-07 20:41:14.981731 Epoch 1

In [18]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)


Accuracy train: 0.82
Accuracy val: 0.62


Part B Dropout Activation

In [19]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
      super().__init__()
      self.n_chans1 = n_chans1
      self.conv1 = nn.Sequential(
        *(n_blocks * [NetRes(n_chans1=n_chans1)]))
      self.conv1_dropout = nn.Dropout2d(p=0.3)
      self.fc1 = nn.Linear(8*8*n_chans1 // 2, 32)
      self.fc2 = nn.Linear(32,2)
        
    def forward(self, x):
      out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
      out = self.conv1_dropout(out)
      out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
      out = self.conv2_dropout(out)
      out = out.view(-1, 8*8*self.n_chans1 // 2)
      out = torch.tanh(self.fc1(out))
      out = self.fc2(out)
      return out

In [20]:
train_loader = torch.utils.data.DataLoader(cifar10, 
                        batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)


2022-12-07 21:39:34.725224 Epoch 1, Training loss 2.0523412044700757
2022-12-07 21:39:46.803430 Epoch 2, Training loss 1.8290732533425627
2022-12-07 21:39:58.869042 Epoch 3, Training loss 1.6545825513732402
2022-12-07 21:40:11.089745 Epoch 4, Training loss 1.5343294533927117
2022-12-07 21:40:23.203323 Epoch 5, Training loss 1.4583565065318056
2022-12-07 21:40:35.283930 Epoch 6, Training loss 1.390545995491545
2022-12-07 21:40:47.388342 Epoch 7, Training loss 1.3248967944508623
2022-12-07 21:40:59.474130 Epoch 8, Training loss 1.2681890617093772
2022-12-07 21:41:11.663181 Epoch 9, Training loss 1.225583272135776
2022-12-07 21:41:23.729705 Epoch 10, Training loss 1.1904671711994863
2022-12-07 21:41:35.827664 Epoch 11, Training loss 1.159213604677059
2022-12-07 21:41:47.900320 Epoch 12, Training loss 1.1327191531048406
2022-12-07 21:41:59.933491 Epoch 13, Training loss 1.108315679956885
2022-12-07 21:42:12.079786 Epoch 14, Training loss 1.0868111956302466
2022-12-07 21:42:24.146927 Epoch 

In [21]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)


Accuracy train: 0.82
Accuracy val: 0.62


Part B, weight decay

In [22]:
def training_loop(n_epochs, optimizer, model, loss_fn, 
                  train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs.to('cuda:0'))
      loss = loss_fn(outputs.to('cuda:0'), 
                     labels.to('cuda:0'))
      
      ambda = 0.001
      norm = sum(p.pow(2.0).sum()
                    for p in model.parameters())
    
      loss = loss + ambda*norm

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
     
    print('{} Epoch {}, Training Loss {}'.format(datetime.datetime.now(),
                                    epoch, loss_train / len(train_loader)))


In [25]:
train_loader = torch.utils.data.DataLoader(cifar10, 
                    batch_size=64, shuffle=True)

model = Net().to('cuda:0')
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs =300,
    optimizer= optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    
)

2022-12-07 22:58:12.849836 Epoch 1, Training Loss 2.0841912031173706
2022-12-07 22:58:26.036678 Epoch 2, Training Loss 1.8432978106581646
2022-12-07 22:58:39.258628 Epoch 3, Training Loss 1.6726413815832504
2022-12-07 22:58:52.378705 Epoch 4, Training Loss 1.5799622424423236
2022-12-07 22:59:05.474073 Epoch 5, Training Loss 1.5235693555353853
2022-12-07 22:59:18.584566 Epoch 6, Training Loss 1.4768045788530804
2022-12-07 22:59:31.614084 Epoch 7, Training Loss 1.432058575817996
2022-12-07 22:59:44.675516 Epoch 8, Training Loss 1.3874695857467554
2022-12-07 22:59:57.645479 Epoch 9, Training Loss 1.3414176331304224
2022-12-07 23:00:10.691019 Epoch 10, Training Loss 1.3009408210092188
2022-12-07 23:00:23.744659 Epoch 11, Training Loss 1.2690469714076928
2022-12-07 23:00:36.918254 Epoch 12, Training Loss 1.2431152326524104
2022-12-07 23:00:49.998278 Epoch 13, Training Loss 1.2232147888149447
2022-12-07 23:01:02.865343 Epoch 14, Training Loss 1.2052133106209737
2022-12-07 23:01:15.891741 Epo

In [26]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                          shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                        shuffle=False)
validate(model, train_loader, val_loader)


Accuracy train: 0.76
Accuracy val: 0.68
